# Task 2: Lookalike Model


In [29]:
import pandas as pd

### **Load the datasets**

In [30]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

### **Merge transaction data with customer and product data**

In [45]:
customer_transactions_df = pd.merge(transactions_df, customers_df, on='CustomerID', how='inner')
customer_transactions_df = pd.merge(customer_transactions_df, products_df, on='ProductID', how='inner')

### **Preview the data**

In [33]:
customer_transactions_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


### **Aggregate data by customer to create features**

In [34]:
customer_summary = customer_transactions_df.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    total_products_bought=('Quantity', 'sum'),
    frequent_category=('Category', lambda x: x.mode()[0]),
    last_purchase_date=('TransactionDate', 'max')
).reset_index()

### **Add recency feature: Time since last purchase (in days)**

In [ ]:
customer_summary['recency'] = (pd.to_datetime('today') - pd.to_datetime(customer_summary['last_purchase_date'])).dt.days

### **Display the customer summary with features**

In [14]:
customer_summary.head()

,CustomerID,total_spend,total_products_bought,frequent_category,last_purchase_date,recency
0,C0001,3354.52,12,Electronics,2024-11-02 17:04:16,86
1,C0002,1862.74,10,Clothing,2024-12-03 01:41:41,55
2,C0003,2725.38,14,Home Decor,2024-08-24 18:54:04,155
3,C0004,5354.88,23,Books,2024-12-23 14:13:52,35
4,C0005,2034.24,7,Electronics,2024-11-04 00:30:22,84


### **Aggregate data by customer to create features, including Region**

In [35]:
customer_summary = customer_transactions_df.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    total_products_bought=('Quantity', 'sum'),
    frequent_category=('Category', lambda x: x.mode()[0]),
    last_purchase_date=('TransactionDate', 'max')
).reset_index()

### **Merge back the 'Region' column from customers_df**

In [36]:
customer_summary = pd.merge(customer_summary, customers_df[['CustomerID', 'Region']], on='CustomerID', how='left')

### **Add recency feature: Time since last purchase (in days)**

In [37]:
customer_summary['recency'] = (pd.to_datetime('today') - pd.to_datetime(customer_summary['last_purchase_date'])).dt.days

### **Display the customer summary with features including Region**

In [38]:
customer_summary.head()

,CustomerID,total_spend,total_products_bought,frequent_category,last_purchase_date,Region,recency
0,C0001,3354.52,12,Electronics,2024-11-02 17:04:16,South America,86
1,C0002,1862.74,10,Clothing,2024-12-03 01:41:41,Asia,55
2,C0003,2725.38,14,Home Decor,2024-08-24 18:54:04,South America,156
3,C0004,5354.88,23,Books,2024-12-23 14:13:52,South America,35
4,C0005,2034.24,7,Electronics,2024-11-04 00:30:22,Asia,84


### **Apply transformations: Standardize numerical features, OneHotEncode categorical ones**

In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

### **Fit the transformations and apply them**

In [40]:
X = preprocessor.fit_transform(customer_summary)

### **Calculate the cosine similarity matrix**

In [42]:
cosine_sim = cosine_similarity(X)

In [43]:
def get_top_3_similar(customers_df, cosine_sim):
    lookalike_map = {}
    # Get the first 20 customers from customers_df
    first_20_customers = customers_df.head(20)
    # Loop over the first 20 customers and print their CustomerID
    for i, customer_id in enumerate(first_20_customers['CustomerID']):
        # Get similarity scores for the current customer (excluding self-comparison)
        similarity_scores = list(enumerate(cosine_sim[i]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:4]  # Exclude self
        # Create a list of the top 3 customers and their similarity scores
        top_3_customers = [(customers_df['CustomerID'][idx], score) for idx, score in similarity_scores]
        lookalike_map[customer_id] = top_3_customers
    return lookalike_map

### **Get the top 3 lookalikes for each customer**

In [48]:
lookalike_map = get_top_3_similar(customer_summary, cosine_sim)

### **Convert lookalike map to DataFrame and save to CSV**

In [49]:
lookalike_df = pd.DataFrame([(key, value) for key, value in lookalike_map.items()], columns=['CustomerID', 'Lookalikes'])

### **Preview the first 20 lookalikes**

In [50]:
lookalike_df.head(20)

,CustomerID,Lookalikes
0,C0001,"[(C0048, 0.9430113822024848), (C0190, 0.904352..."
1,C0002,"[(C0088, 0.992653988273535), (C0092, 0.9354717..."
2,C0003,"[(C0052, 0.9191981132314003), (C0025, 0.832750..."
3,C0004,"[(C0169, 0.980271934704057), (C0165, 0.9722075..."
4,C0005,"[(C0186, 0.993371250274052), (C0140, 0.9802782..."
5,C0006,"[(C0187, 0.9854786287526796), (C0168, 0.955264..."
6,C0007,"[(C0140, 0.8901442000178618), (C0186, 0.871736..."
7,C0008,"[(C0065, 0.84998286628516), (C0059, 0.78591301..."
8,C0009,"[(C0198, 0.9989885857351463), (C0103, 0.933237..."
9,C0010,"[(C0111, 0.9167406175011905), (C0061, 0.861985..."


### **Save to CSV**

In [51]:
lookalike_df.to_csv('Lookalike.csv', index=False)